In [66]:
import numpy as np
import pandas as pd
from collections import Counter

In [170]:
# Generate a feature matrix in which each column consists in the 20 features representing an item in the task
def item_features(Nitems,g):
    return np.random.geometric(g,(20,Nitems))

# Compute the episodic Matrix which is a noisy representation of the feature matrix related to the target items
def episodic_matrix(wT,g,u,c):
    M = wT*0 # initialize M. wT is the feature matrix representing the study targets
    for i in range(len(wT[0])): # for each column in M
        copy = np.random.binomial(1,u,20) # given an item, each feature has a probability u to be copied in M
        trace = copy*wT[:,i] # some features of an item in wT are copied, providing a trace
        correct = np.random.binomial(1,c,20) # some of the features copied are effectively copied correctly
        wrong_copied = np.where((trace>0) * (correct==0)) # position of the feature which are copied non correctly in the trace
        trace[wrong_copied] = np.random.geometric(g,len(wrong_copied)) # Random features substitute the wrong copied original features
        M[:,i] = trace # a final trace is stored in the specific column of the episodic matrix
        # M will be similar to the feature matrix for the study targets except for some elements (elements not copied or wrong copied)
    return M

# Compute the nonzero mismatches between a probe (a presented item in the task) and a trace
def qtype(probe,trace):
    # extract the nonzero elements from the trace to count only the nonzero mismatches
    nonzero_probe = probe[trace > 0]
    nonzero_trace = trace[trace > 0]
    # sum mismatches 
    return sum(~(nonzero_probe == nonzero_trace))

# Compute the nonzero matches between features in the probe and in the trace, and for each feature values
def mtype(probe,trace):
    m = np.tile(0,200) # initialize a table of matches count for each possible feature value from 1 to 200
    # extract the nonzero elements from the trace to count only the nonzero matches
    nonzero_probe = probe[trace > 0]
    nonzero_trace = trace[trace > 0]
    nonzero_match = nonzero_probe[nonzero_probe==nonzero_trace] # paired the matches between probe and trace
    nonzero_match_count = pd.value_counts(nonzero_match) # compute a frequency table of matches for each feature
    m[np.array(nonzero_match_count.keys().tolist(),dtype=np.int32)-1] = nonzero_match_count.tolist() # write the frequency of matched features in m such that posisiton i contains the frequency of matches for feature i
    
    return m

# Compute the similarity between the probe and the trace feature vectors
def similarity(probe,trace,c,g):
    Qtype = qtype(probe,trace)
    Mtype = mtype(probe,trace)
    x1 = (1-c)**Qtype
    x2 = 1
    for i in range(len(Mtype)):
        x2 = x2 * ((c+(1-c)*g*(1-g)**(i))/(0.0001+g*(1-g)**(i)))**Mtype[i] # product over 200 elements
    
    return(x1*x2)

# Compute the decision rule for a given probe (target or distractor) based on the episodic matrix and the similarity
def memory_strength(probe,M,c,g):
    phi = np.zeros(len(M[0]))
    for i in range(len(M[0])):
        phi[i] = similarity(probe,M[:,i],c,g)
    # for each item in the task compute a similarity between the item and the episodic matrix
    return np.mean(phi)

# Generate a binary response for each item (target and distractor)
def response_vector(g,u,c,Nitems):
    # STUDY PHASE #
    W = item_features(Nitems,g) # Generate al the items in the task
    wT = W[:,0:int(Nitems/2)] # Extract the study items
    wD = W[:,int(Nitems/2):Nitems] # Extract the distractor items
    M = episodic_matrix(wT,g,u,c) # Compute the episodic memory based on the study items
    # RECOGNITION PHASE #
    target = np.array([memory_strength(wT[:,i],M,c,g) for i in range(int(Nitems/2))]) # compute phi for each target
    distractor = np.array([memory_strength(wD[:,i],M,c,g) for i in range(int(Nitems/2))]) # compute phi for each distractor
    # Apply the decision rule
    target = target > 1
    distractor = distractor > 1
    target = target.astype(int)
    distractor = distractor.astype(int)
    
    return np.c_[target,distractor]


def REM(pars,Nitems,Ntrials,flag):
    # pars:[g,u,c]
    # g: Environmental Base Rate
    # u: Feature Copying
    # c: Correct Feature Copying
    # Ntimes: targets = Nitems/2; distractors = Nitems/2
    # flag: 1, full matrix output; 2, contingency table
    g,u,c = pars
    out1 = np.zeros((Ntrials,int(Nitems/2),2))
    out2 = np.zeros((Ntrials,2,2))
    for n in range(Ntrials):
        o = response_vector(g,u,c,Nitems)
        out1[n,:,:] = o
        out2[n,:,:] = np.array([[sum(o[:,0]),(Nitems/2)-sum(o[:,0])],
                                [sum(o[:,1]),(Nitems/2)-sum(o[:,1])]])
    if flag:
        # first column: vector of Hit (1) and Miss (0) for targets
        # second column: vector of Correct Rejection (0) and False Alarms (1) for distractors
        return out1
    else:
        # first row (target): Hit and Miss
        # second row (distractor): False Alarms and Correct Rejection
        return out2
                            
    

In [187]:
REM([0.4,0.4,0.6],40,1,0)

array([[[14.,  6.],
        [ 4., 16.]]])